<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=9b993a6bff0aa5d247814c0ef90549c7f6d7897c95805b9d04dbb4669da6104d
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-07 10:33:27
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.45 C
-------------------
Today PnL: 25.89 K (0.18%)
Current PnL: -24.51 L (-15.63%)
CY Booked + Current PnL: -10.22 L (-6.52%)
-------------------
Total profit:  2.29 L
Total loss:  -26.80 L
-------------------
Total Booked + Current PnL: 16.76 L (12.91%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (9.88%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 89.48 L (61.89%)
Deployed:  1.30 C
Current:  1.45 C
CAGR/XIRR %: 5.9%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1530.00,0.02,16.29,2.93,19.69,6533.0,31233.0,222975.0,5.00,71.0,X-MC,4.06,78.0,4.78,1.54,24.19,XY25,NTT,AC
51,NESTLEIND,1377.00,0.45,16.89,3.85,21.40,11945.0,44835.0,310261.0,8.49,72.0,X-LC,5.29,12.0,3.75,2.15,24.29,XY25,NTT,FMCG
49,MASFIN,398.61,0.47,4.01,17.34,22.05,17671.0,3930.0,101910.0,-10.12,65.0,H-SC,6.99,164.0,0.22,0.71,48.13,XR,ATH,FINANCE
77,TTKPRESTIG,770.00,-1.31,-20.71,26.24,0.09,20967.0,-20874.0,79903.0,85.49,38.0,M-SC,3.20,253.0,-1.00,0.55,4.98,OX40N,NTT,DURABLES
17,COALINDIA,484.83,0.22,5.25,13.28,19.23,21030.0,7900.0,158360.0,24.27,82.0,L-LC,6.26,179.0,0.38,1.10,27.64,XY25,ATH,MINING


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAELXSI,9161.00,3.85,-20.01,65.05,32.03,68602.0,-26375.0,105460.0,-13.35,65.0,H-SC,7.08,148.0,-0.38,0.73,18.12,OX40N,NTT,IT
44,JCHAC,2282.00,2.71,-35.32,54.60,-0.01,45133.0,-45143.0,82662.0,16813.04,47.0,M-SC,14.15,233.0,-1.00,0.57,5.80,OX40N,BTT,AC
39,INDIGOPNTS,1408.00,2.65,-13.19,15.21,0.02,23049.0,-23019.0,151540.0,131.03,58.0,M-SC,7.51,234.0,-1.00,1.05,31.81,OX40N,NTT,PAINTS
23,DMART,5391.45,2.63,-9.62,43.39,29.60,97888.0,-24000.0,225600.0,-22.96,44.0,X-LC,9.74,19.0,-0.25,1.56,10.46,X40N,ATH,FMCG
37,IEX,219.00,2.25,3.16,44.07,48.63,90907.0,6324.0,206278.0,-31.26,69.0,H-SC,13.91,136.0,0.07,1.43,15.15,XR,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJOOENG,143.1,-6.27,-28.95,96.48,39.61,70266.0,-29670.0,72830.0,-52.00,58.0,H-SC,31.73,135.0,-0.42,0.50,17.89,AR,ATH,MISC
75,TMPV,600.0,-1.91,-25.84,65.81,22.98,110256.0,-58361.0,167537.0,-23.08,49.0,X-LC,4.73,3.0,-0.53,1.16,5.37,XY24,NTT,AUTO
47,KANSAINER,340.0,-1.39,-20.93,43.51,13.47,92776.0,-56439.0,213228.0,-67.53,54.0,H-SC,5.04,158.0,-0.61,1.48,8.42,XY24,NTT,PAINTS
77,TTKPRESTIG,770.0,-1.31,-20.71,26.24,0.09,20967.0,-20874.0,79903.0,85.49,38.0,M-SC,3.20,253.0,-1.00,0.55,4.98,OX40N,NTT,DURABLES
69,SURYODAY,216.0,-1.21,-18.81,51.54,23.03,74935.0,-33680.0,145391.0,57.89,54.0,H-SC,5.19,167.0,-0.45,1.01,44.26,XR,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.17,-0.8,109.6,107.91,160518.0,-1186.0,146458.0,-19.34,57.0,M-SC,10.15,216.0,-0.01,1.01,9.2,OX40N,ATH,CABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.17,-0.80,109.60,107.91,160518.0,-1186.0,146458.0,-19.34,57.0,M-SC,10.15,216.0,-0.01,1.01,9.20,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,2.65,-13.19,15.21,0.02,23049.0,-23019.0,151540.0,131.03,58.0,M-SC,7.51,234.0,-1.00,1.05,31.81,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,3.85,-20.01,65.05,32.03,68602.0,-26375.0,105460.0,-13.35,65.0,H-SC,7.08,148.0,-0.38,0.73,18.12,OX40N,NTT,IT
77,TTKPRESTIG,770.00,-1.31,-20.71,26.24,0.09,20967.0,-20874.0,79903.0,85.49,38.0,M-SC,3.20,253.0,-1.00,0.55,4.98,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.39,-20.93,43.51,13.47,92776.0,-56439.0,213228.0,-67.53,54.0,H-SC,5.04,158.0,-0.61,1.48,8.42,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,0.26,3.91,16.19,20.74,34180.0,7948.0,211118.0,38.17,74.0,M-MC,6.31,189.0,0.23,1.46,14.94,XY25,NTT,FINANCE
17,COALINDIA,484.83,0.22,5.25,13.28,19.23,21030.0,7900.0,158360.0,24.27,82.0,L-LC,6.26,179.0,0.38,1.10,27.64,XY25,ATH,MINING
84,WIPRO,420.00,2.00,11.11,55.04,72.27,103627.0,18827.0,188275.0,-4.72,63.0,M-LC,6.08,99.0,0.18,1.30,17.97,XR,NTT,IT
49,MASFIN,398.61,0.47,4.01,17.34,22.05,17671.0,3930.0,101910.0,-10.12,65.0,H-SC,6.99,164.0,0.22,0.71,48.13,XR,ATH,FINANCE
37,IEX,219.00,2.25,3.16,44.07,48.63,90907.0,6324.0,206278.0,-31.26,69.0,H-SC,13.91,136.0,0.07,1.43,15.15,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,MASFIN,398.61,0.47,4.01,17.34,22.05,17671.0,3930.0,101910.0,-10.12,65.0,H-SC,6.99,164.0,0.22,0.71,48.13,XR,ATH,FINANCE
84,WIPRO,420.00,2.00,11.11,55.04,72.27,103627.0,18827.0,188275.0,-4.72,63.0,M-LC,6.08,99.0,0.18,1.30,17.97,XR,NTT,IT
37,IEX,219.00,2.25,3.16,44.07,48.63,90907.0,6324.0,206278.0,-31.26,69.0,H-SC,13.91,136.0,0.07,1.43,15.15,XR,NTT,MISC
25,FINCABLES,1641.55,0.17,-0.80,109.60,107.91,160518.0,-1186.0,146458.0,-19.34,57.0,M-SC,10.15,216.0,-0.01,1.01,9.20,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,0.78,-1.05,37.07,35.63,76874.0,-2199.0,207375.0,-14.97,59.0,H-MC,2.88,119.0,-0.03,1.43,15.45,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,0.28,-15.84,31.62,10.78,63956.0,-38055.0,202263.0,-49.73,14.0,X-LC,4.98,5.0,-0.60,1.40,0.28,X40,NTT,FMCG
7,AWL,485.00,0.11,-25.11,109.90,57.19,268409.0,-81897.0,244230.0,-64.65,30.0,X-MC,8.40,58.0,-0.31,1.69,0.11,XY24,NTT,FMCG
22,DIXON,18931.72,1.19,-17.73,59.55,31.26,113059.0,-40912.0,189856.0,-61.61,31.0,X-MC,18.14,56.0,-0.36,1.31,1.19,X40N,ATH,IT
52,PAGEIND,51605.07,0.03,-10.48,45.90,30.62,97409.0,-24834.0,212220.0,-36.35,33.0,X-MC,16.01,55.0,-0.25,1.47,0.03,X40,ATH,APPARELS
11,BATAINDIA,2096.00,-0.95,-39.56,123.72,35.20,96208.0,-50907.0,77763.0,1.95,36.0,X-SC,17.25,92.0,-0.53,0.54,0.00,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1644.00,-0.48,0.64,20.00,20.77,47128.0,1496.0,235640.0,-8.06,38.0,X-MC,0.97,62.0,0.03,1.63,7.60,X40N,NTT,BREWERIES
65,SIEMENS,4671.50,-0.54,-16.41,50.15,25.51,78016.0,-30530.0,155565.0,-4.56,48.0,H-LC,1.43,49.0,-0.39,1.08,14.64,AR,ATH,ELECTRICAL
20,DABUR,735.00,0.50,2.24,40.55,43.70,102847.0,5563.0,253631.0,-6.13,67.0,X-MC,2.02,72.0,0.05,1.75,17.63,XY24,BTT,FMCG
81,VBL,671.64,0.58,1.35,33.86,35.67,108391.0,4274.0,320116.0,-10.77,67.0,X-LC,2.13,4.0,0.04,2.21,15.40,X40N,ATH,FMCG
45,JIOFIN,387.00,0.40,-3.58,29.11,24.48,75041.0,-9580.0,257785.0,-5.23,52.0,X-LC,2.26,36.0,-0.13,1.78,49.42,XY24,BTT,FINANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.95,-39.56,123.72,35.20,96208.0,-50907.0,77763.0,1.95,36.0,X-SC,17.25,92.0,-0.53,0.54,0.00,X40,NTT,FOOTWEAR
52,PAGEIND,51605.07,0.03,-10.48,45.90,30.62,97409.0,-24834.0,212220.0,-36.35,33.0,X-MC,16.01,55.0,-0.25,1.47,0.03,X40,ATH,APPARELS
7,AWL,485.00,0.11,-25.11,109.90,57.19,268409.0,-81897.0,244230.0,-64.65,30.0,X-MC,8.40,58.0,-0.31,1.69,0.11,XY24,NTT,FMCG
43,ITC,452.00,0.28,-15.84,31.62,10.78,63956.0,-38055.0,202263.0,-49.73,14.0,X-LC,4.98,5.0,-0.60,1.40,0.28,X40,NTT,FMCG
58,RELAXO,1176.00,-0.93,-48.12,193.56,52.31,145776.0,-69847.0,75313.0,-44.73,42.0,X-SC,7.61,91.0,-0.48,0.52,0.81,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,0.45,-25.99,93.01,42.84,44747.0,-16896.0,48110.0,-51.52,58.0,X-SC,28.53,83.0,-0.38,0.33,8.84,XY24,NTT,MISC
58,RELAXO,1176.00,-0.93,-48.12,193.56,52.31,145776.0,-69847.0,75313.0,-44.73,42.0,X-SC,7.61,91.0,-0.48,0.52,0.81,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.95,-39.56,123.72,35.20,96208.0,-50907.0,77763.0,1.95,36.0,X-SC,17.25,92.0,-0.53,0.54,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,2.09,3.05,20.40,24.08,26688.0,3875.0,130825.0,-5.33,59.0,X-SC,6.72,89.0,0.15,0.91,23.27,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.03,-16.09,70.81,43.32,96769.0,-26212.0,136660.0,-27.94,53.0,X-SC,7.77,90.0,-0.27,0.95,4.93,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.64,-9.91,33.98,20.71,115789.0,-37481.0,340756.0,-22.33,58.0,X-LC,2.66,1.0,-0.32,2.36,13.86,X40,ATH,IT
41,INFY,1972.00,1.24,11.11,20.82,34.24,72383.0,34771.0,347659.0,-13.50,55.0,X-LC,7.54,2.0,0.48,2.41,20.32,X40,NTT,IT
75,TMPV,600.00,-1.91,-25.84,65.81,22.98,110256.0,-58361.0,167537.0,-23.08,49.0,X-LC,4.73,3.0,-0.53,1.16,5.37,XY24,NTT,AUTO
81,VBL,671.64,0.58,1.35,33.86,35.67,108391.0,4274.0,320116.0,-10.77,67.0,X-LC,2.13,4.0,0.04,2.21,15.40,X40N,ATH,FMCG
43,ITC,452.00,0.28,-15.84,31.62,10.78,63956.0,-38055.0,202263.0,-49.73,14.0,X-LC,4.98,5.0,-0.60,1.40,0.28,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.83,-33.06,98.53,32.89,50919.0,-25528.0,51679.0,-738.30,69.0,L-MC,7.52,259.0,-0.50,0.36,42.32,XR,NTT,BANKS
49,MASFIN,398.61,0.47,4.01,17.34,22.05,17671.0,3930.0,101910.0,-10.12,65.0,H-SC,6.99,164.0,0.22,0.71,48.13,XR,ATH,FINANCE
13,BSOFT,831.70,2.09,-22.02,93.37,50.79,102404.0,-30977.0,109676.0,0.12,53.0,H-SC,10.77,151.0,-0.30,0.76,27.70,XR,ATH,IT
69,SURYODAY,216.00,-1.21,-18.81,51.54,23.03,74935.0,-33680.0,145391.0,57.89,54.0,H-SC,5.19,167.0,-0.45,1.01,44.26,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,2.65,-13.19,15.21,0.02,23049.0,-23019.0,151540.0,131.03,58.0,M-SC,7.51,234.0,-1.00,1.05,31.81,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-1.20,-27.02,54.71,12.90,87237.0,-59043.0,159453.0,134.38,70.0,H-SC,10.60,161.0,-0.68,1.10,32.58,OX40N,NTT,FINANCE
17,COALINDIA,484.83,0.22,5.25,13.28,19.23,21030.0,7900.0,158360.0,24.27,82.0,L-LC,6.26,179.0,0.38,1.10,27.64,XY25,ATH,MINING
82,VOLTAS,1530.00,0.02,16.29,2.93,19.69,6533.0,31233.0,222975.0,5.00,71.0,X-MC,4.06,78.0,4.78,1.54,24.19,XY25,NTT,AC
36,ICICIPRULI,790.00,0.57,14.58,14.10,30.73,31827.0,28717.0,225722.0,-12.21,78.0,X-MC,6.17,75.0,0.90,1.56,29.21,X40,ATH,INSURANCE
57,RECLTD,446.00,0.26,3.91,16.19,20.74,34180.0,7948.0,211118.0,38.17,74.0,M-MC,6.31,189.0,0.23,1.46,14.94,XY25,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.17
1,25,44.15
2,50,76.43


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.87
MC    30.78
LC    26.33
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.41
X40      23.87
X40N     12.73
XY25     10.69
XR        9.12
AR        8.63
OX40N     7.58
SR        0.95
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.29
H-SC    22.94
X-LC    22.85
M-SC    11.27
X-SC     7.88
H-MC     4.67
M-MC     1.46
M-LC     1.30
L-LC     1.10
H-LC     1.08
L-SC     0.78
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.97,-7.31,42.52
IT,12.76,-15.90,74.35
FINANCE,11.16,-10.54,56.94
MISC,7.19,-24.96,76.08
ELECTRICAL,5.93,-9.58,49.71
PAINTS,5.72,-11.68,28.27
INSURANCE,4.70,0.34,33.77
PHARMA,4.00,-1.38,33.29
AUTO,2.81,-29.14,73.91


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3152848.0,21
AR,1322785.0,10
XR,1176128.0,12
X40,1069295.0,15
X40N,901027.0,9
OX40N,694677.0,10
XY25,347966.0,7
SR,282940.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3481277.0,24
X-MC,1470039.0,16
M-SC,1428519.0,15
X-LC,919519.0,13
X-SC,780240.0,8
H-MC,402799.0,3
L-SC,177501.0,2
M-LC,103627.0,1
H-LC,78016.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1230720.0      6
           AR         910912.0      5
M-SC       XY24       819799.0      6
H-SC       XR         753173.0      7
X-MC       X40        474976.0      7
           XY24       435006.0      3
X-LC       X40        406722.0      6
X-MC       X40N       388522.0      4
X-SC       X40N       306226.0      3
H-SC       OX40N      303532.0      4
M-SC       OX40N      300447.0      5
X-SC       XY24       286417.0      3
H-SC       SR         282940.0      2
H-MC       AR         207190.0      2
X-LC       X40N       206279.0      2
H-MC       XY24       195609.0      1
X-SC       X40        187597.0      2
X-LC       XY24       185297.0      2
M-SC       XR         181606.0      2
X-MC       XY25       171535.0      2
M-SC       AR         126667.0      2
X-LC       XY25       121221.0      3
M-LC       XR         103627.0      1
L-SC       OX40N       90698.0      1
           XR          86803.0      1
H-LC       AR          78016.0      1
L-MC       XR          50919.0      1
M-MC       XY25        34180.0      1
L-LC       XY25        21030.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
